In [51]:
%%configure -f
{"name": "brian-load-modelcubes", "executorMemory": "25G", "numExecutors": 8, "executorCores": 2,
 "conf": {"spark.yarn.appMasterEnv.PYSPARK_PYTHON":"python3"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
154,application_1588740809550_0148,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
153,application_1588740809550_0147,pyspark,idle,Link,Link,
154,application_1588740809550_0148,pyspark,idle,Link,Link,✔


In [52]:
import sys
import os
import subprocess
from io import BytesIO
from gzip import GzipFile
from pyspark.sql import Row
import glob
import time
import numpy as np
from itertools import repeat

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
def get_fits_module():
    if 'astropy' not in sys.modules:
        stdout = subprocess.check_output(
            sys.executable + ' -m pip install astropy',
            stderr=subprocess.STDOUT,
            shell=True).decode('utf-8')
    from astropy.io import fits
    return fits

def getfitslocal(path):
    fits = get_fits_module()
    fits_obj = fits.open(path)
    return fits_obj

# Spark dataframe cannot do numpy types
def typeconv(i):
    try:
        return i.item()
    except:
        return i
    
def createPerPixelSpaxelRow(fits):
    n_wave, n_y, n_x = fits['FLUX'].data.shape
    
    hdr = fits[0].header
    flux = np.ravel(fits['FLUX'].data)
    ivar = np.ravel(fits['IVAR'].data)
    mask = np.ravel(fits['MASK'].data)
    model = np.ravel(fits['MODEL'].data)
    emline = np.ravel(fits['EMLINE'].data)
    emline_base = np.ravel(fits['EMLINE_BASE'].data)
    emline_mask = np.ravel(fits['EMLINE_MASK'].data)
    
    wave = fits['WAVE'].data
    
    binid = np.ravel(fits['BINID'].data[0])

    # ravel indexing
    # rav_idx = (wave_idx * n_y * n_y) + (y_idx * n_x) + x_idx

    # and back
    # wave_idx = int(rav_idx / (n_y * n_y))
    # spaxel_idx = rav_idx - wave_idx * (n_y * n_y)
    # y_idx = int(spaxel_idx / n_y)
    # x_idx = spaxel_idx - y_idx * n_y
    
    rows = []
    for ravel_idx, flux_val in enumerate(flux):
        wave_idx = int(ravel_idx / (n_y * n_y))
        spaxel_idx = ravel_idx - wave_idx * (n_y * n_y)
        y_idx = int(spaxel_idx / n_y)
        x_idx = spaxel_idx - y_idx * n_y
        
        # skip no good spaxels
        if binid[spaxel_idx] == -1:
            continue

        row = {'plateifu': typeconv(hdr.get('PLATEIFU')),
               'mangaid': typeconv(hdr.get('MANGAID')),
               'ra': typeconv(hdr.get('OBJRA')),
               'dec': typeconv(hdr.get('OBJDEC')),
               'daptype': typeconv(hdr.get('DAPTYPE')),
               'spaxel_idx': spaxel_idx,
               'x': x_idx,
               'y': y_idx,
               'wave': typeconv(wave[wave_idx]),
               'flux': typeconv(flux_val),
               'ivar': typeconv(ivar[ravel_idx]),
               'mask': typeconv(mask[ravel_idx]),
               'model': typeconv(model[ravel_idx]),
               'emline': typeconv(emline[ravel_idx]),
               'emline_base': typeconv(emline_base[ravel_idx]),
               'emline_mask': typeconv(emline_mask[ravel_idx]),
               'binid_binned_spectra': typeconv(binid[spaxel_idx]),
               'binid_stellar_cont': typeconv(np.ravel(fits['BINID'].data[1])[spaxel_idx]),
               'binid_emline_moments': typeconv(np.ravel(fits['BINID'].data[2])[spaxel_idx]),
               'binid_emline_models': typeconv(np.ravel(fits['BINID'].data[3])[spaxel_idx]),
               'binid_spectral_indices': typeconv(np.ravel(fits['BINID'].data[4])[spaxel_idx])
              }
            
        rows.append(Row(**row))
    return rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
from itertools import repeat

def createPerSpaxelRow(fits):
    n_wave, n_y, n_x = fits['FLUX'].data.shape
    n_spax = np.int(n_y * n_x)
     
    mapRow = Row('plateifu', 'mangaid', 'ra', 'dec', 'daptype', 'spaxel_idx', 'x', 'y', 
                 'wave', 'flux', 'ivar', 'mask', 'model', 'emline', 'emline_base', 
                 'emline_mask', 'binid_binned_spectra', 'binid_stellar_cont', 
                 'binid_emline_moments', 'binid_emline_models', 'binid_spectral_indices')
    
    hdr = fits[0].header
    plateifu = repeat(hdr.get('PLATEIFU'))
    mangaid = repeat(hdr.get('MANGAID'))
    ra = repeat(hdr.get('OBJRA'))
    dec = repeat(hdr.get('OBJDEC'))
    daptype = repeat(hdr.get('DAPTYPE'))

    flux = np.reshape(fits['FLUX'].data, (n_wave, n_spax))
    ivar = np.reshape(fits['IVAR'].data, (n_wave, n_spax))
    mask = np.reshape(fits['MASK'].data, (n_wave, n_spax))
    model = np.reshape(fits['MODEL'].data, (n_wave, n_spax))
    emline = np.reshape(fits['EMLINE'].data, (n_wave, n_spax))
    emline_base = np.reshape(fits['EMLINE_BASE'].data, (n_wave, n_spax))
    emline_mask = np.reshape(fits['EMLINE_MASK'].data, (n_wave, n_spax))
    
    wave = fits['WAVE'].data.tolist()
    
    binid = np.reshape(fits['BINID'].data, (5, n_spax))

    # ravel indexing
    # rav_idx = (wave_idx * n_y * n_y) + (y_idx * n_x) + x_idx

    # and back
    # wave_idx = int(rav_idx / (n_y * n_y))
    # spaxel_idx = rav_idx - wave_idx * (n_y * n_y)
    # y_idx = int(spaxel_idx / n_y)
    # x_idx = spaxel_idx - y_idx * n_y
    
    rows = []
    for spaxel_idx in range(n_spax):
        y_idx = int(spaxel_idx / n_y)
        x_idx = spaxel_idx - y_idx * n_y
        x = repeat(x_idx)
        y = repeat(y_idx)
        
        # skip no good spaxels
        if binid[0, spaxel_idx] == -1:
            continue

        row = {'plateifu': plateifu,
               'mangaid': mangaid,
               'ra': ra,
               'dec': dec,
               'daptype': daptype,
               'spaxel_idx': repeat(spaxel_idx),
               'x': x,
               'y': y,
               'wave': wave,
               'flux': flux[:,spaxel_idx].tolist(),
               'ivar': ivar[:,spaxel_idx].tolist(),
               'mask': mask[:,spaxel_idx].tolist(),
               'model': model[:,spaxel_idx].tolist(),
               'emline': emline[:,spaxel_idx].tolist(),
               'emline_base': emline_base[:,spaxel_idx].tolist(),
               'emline_mask': emline_mask[:,spaxel_idx].tolist(),
               'binid_binned_spectra': repeat(int(binid[0, spaxel_idx])),
               'binid_stellar_cont': repeat(int(binid[1, spaxel_idx])),
               'binid_emline_moments': repeat(int(binid[2, spaxel_idx])),
               'binid_emline_models': repeat(int(binid[3, spaxel_idx])),
               'binid_spectral_indices': repeat(int(binid[4, spaxel_idx]))
              }
        
        row_gen = zip(*row.values())
        _rows = [mapRow(*i) for i in row_gen]
        rows.extend(_rows)
    return rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
base_dir = '/sciserver/vc/manga/vc/sas/dr15/manga/spectro/analysis/v2_4_3/2.2.1/HYB10-GAU-MILESHC/'
dirs1 = [base_dir + i for i in os.listdir(base_dir)]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
files_rdd = sc.parallelize(dirs1, len(dirs1)).flatMap(lambda x: [x+'/'+i for i in os.listdir(x)]).flatMap(lambda d: glob.glob(d+'/*8485*1901*-LOGCUBE-*.fits.gz'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
files_stats = files_rdd.map(
    lambda x: (os.stat(x).st_size/1024/1024, 1, os.stat(x).st_size/1024/1024)
).reduce(
    lambda x,y: (x[0]+y[0], x[1]+y[1], max(x[2],y[2]))
)
print('N files: {1}. Total Size: {0:0.0f}MB, average size: {2:0.0f}'.format(*files_stats))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

N files: 1. Total Size: 19MB, average size: 19

In [58]:
n_part = int(files_stats[1])
table_data = files_rdd.repartition(n_part).map(getfitslocal).flatMap(createPerSpaxelRow)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
hdfs_dir = 'hdfs:///manga/brian-test/dr15/v2_4_3/small_modelcubes'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
t = time.time()
table = spark.createDataFrame(table_data)
table.write.mode('overwrite').parquet(hdfs_dir)
print(time.time()-t)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50.09906792640686

In [62]:
table_data.first()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(plateifu='8485-1901', mangaid=128, ra='HYB10-GAU-MILESHC', dec=0.0, daptype=15, spaxel_idx=3621.595947265625, x=0.0, y=0.0, wave=33, flux=-0.028015678748488426, ivar=236.5732421875, mask='1-209232', model=48.6902009334, emline=6, emline_base=31, emline_mask=31, binid_binned_spectra=0, binid_stellar_cont=219, binid_emline_moments=0, binid_emline_models=232.544703894, binid_spectral_indices=144)